In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
base_path = '/kaggle/input/commonlitreadabilityprize/'
train_path = os.path.join(base_path, 'train.csv')
test_path = os.path.join(base_path, 'test.csv')

model_base_path = '/kaggle/input/download-data-text/'

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
torch.cuda.empty_cache()
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained('/kaggle/input/download-data-text/', local_files_only=True)
model = model_class.from_pretrained('/kaggle/input/download-data-text/', local_files_only=True)

In [ ]:
def chunks(l, n):
    for i in range(0, len(l), n):
#         print(f'processing batch: {i/n}')
        yield l[i:i + n]

In [ ]:
DEVICE = torch.device("cuda")
model.to(DEVICE)
def proccess_through_tranformer(data, max_len=314):
    tokenized = data['excerpt'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)
    
    input_ids = torch.tensor(padded).to(DEVICE)  
    attention_mask = torch.tensor(attention_mask).to(DEVICE)
    
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
        
    features = last_hidden_states[0][:,0,:].cpu().numpy()
    
    return features
    

In [ ]:
targets = train['target']

In [ ]:
features = []

for data in chunks(train, 64):
    features.extend(proccess_through_tranformer(data))
    

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size=0.1)

lr_clf = LinearRegression()
lr_clf.fit(train_features, train_labels)

y = lr_clf.predict(test_features)
mean_absolute_error(y, test_labels)

In [ ]:
features_test = []

for data in chunks(test, 64):
    features_test.extend(proccess_through_tranformer(data))

In [ ]:
features_test = np.array(features_test)
print(features_test)
print(features_test.shape)
print(type(features_test))
y = lr_clf.predict(features_test)
ids = test['id']

print(y)
print(y.shape)
print(type(y))


In [ ]:
submission_df = pd.DataFrame({'id': test.id, 'target': y})

In [ ]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
submission_df